In [ ]:
###
### This notebook was run in a googlecolab environment using GPU
###

!pip install sentencepiece
!pip install transformers

from itertools import chain
import torch 
from transformers import AutoConfig, AutoModelWithLMHead, AutoTokenizer
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
#import torch_xla
#import torch_xla.core.xla_model as xm
dev = "cuda"

In [ ]:
LM_MODEL_TO_USE = "gpt2"
device = torch.device( dev)
tokenizer = AutoTokenizer.from_pretrained(LM_MODEL_TO_USE)
lm = AutoModelWithLMHead.from_pretrained(LM_MODEL_TO_USE)
lm.eval()

lm.to(device)
for param in lm.parameters():
    param.requires_grad = False

In [ ]:
args = pd.read_csv("args.tsv", sep ="\t")
len(args["conclusion"].unique())

In [ ]:
inputs = []
for inp in args["conclusion"].unique():
    x = torch.tensor(tokenizer.encode(inp)).unsqueeze(0) 
    start = torch.tensor(tokenizer.encode("- What do you think about: " )).unsqueeze(0)
    end = torch.tensor(tokenizer.encode(". Do you agree? \n -The answer is no, because ")).unsqueeze(0)
    if x.size()[1]>78:
        input_ids = input_ids[:,:77]
        print("trunc")
    input_ids = torch.cat((start,x,end),1).to(device) 
    inputs.append(input_ids)

In [ ]:
%%time
y = []
for x in inputs:
    pred = lm.generate(input_ids=x, stepsize=0.03,num_samples=2, temperature=1.7, max_length=250, top_k=10, grad_length=1000, horizon_length=9,
                                                                                          gm_scale=0.9, kl_scale=0.7, repetition_penalty=2.0, gamma=1.5)
    y.append(tokenizer.decode(pred[0]))
pd.DataFrame({"conclusion": list(args["conclusion"].unique()), "negative": y}).to_csv("gpt2_negative.csv", "\t", index=False)
